# Neuronale Netze mit Python und TensorFlow

Der vorherige Beitrag *Link* ist eine Einführung in TensorFlow und eine Sammlung von relevante Links. Da findet man auch eine Erklärung wie man TensorFlow installieren kann. 

In diesem Blogbeitrag werden wir einen Überblick über Neuronale Netze bekommen und wir werden eins implementieren. Es ist empfohlen, dass man sich den vorherigen Blogbeitrag anschaut, um besseren Überblick über TensorFlow zu bekommen.

## Echte Neuronale Netzwerke

*Bild von biologischen Neuron*

Das ist ein Neuron. Im menschlichen Gehird gibt es 86 Milliarden davon. Die Dendriten bilden Kontaktstellen für andere Zellen, deren Erregung hier auf die Nervenzelle übertragen werden kann. 

**TODO: Continue Description**

## Künstliche Neuronale Netz

Perzeptronen und Sigmoidneurone sind die Hauptbestandteile eines neuronalen Netzes. 

*Bild von einem Perzeptron*

Ähnlich wie ein Neuron, haben Perzeptronen mehrere **Inputs** (*x*) mit entsprechende **Gewichtungen** (*w*). Die Inputs sind immer 0 oder 1. Jedes Input wird mit einer der Gewichtungen multipliziert und man addiert noch ein **Bias** (*b*) dazu. Am Ende summiert man alle Ergebnisse und verwendet einen **Schwellwert** um zu entscheiden, ob 1 oder 0 ausgegeben wird.

**TODO: 'Bias' in German**

Perzeptronen sind oft zu primitiv für künstliche neuronale Netze, deswegen benutzt man Sigmoidneuronen. Die Unterschied ist, dass sie auch eine **Aktivierungsfunktion** haben. 

*Bild von Sigmoidfunktion*

Diese Funktion erlaubt, dass diese Sigmoidneuronen als Inputs Zahlen zwischen 0 und 1 bekommen. Das hilft so, dass wenn man kleine Anpassungen an den Gewichtungen und Biasen macht, dann gibt es auch kleine Unterschiede bei der Ausgabe.

Ein neuronales Netz besteht aus mehrere Schichten und jede Schicht hat mehrere Neuronen, wobei in der Regel hat jedes Neuron aus einer Schicht eine Verbindung zu allen anderen Neuronen aus der nächste Schicht. Die erste Schicht ist die Inputschicht, wo man sein Datensatz eingibt. Es kann beliebig viele Zwischenschichten geben, aber die letzte ist die Outputschicht, mit 1 oder mehreren Neuronen (Die Anzahl hängt vom Problem, den man lösen möchte ab). 

## Implementierung

Wir werden ein eifaches neuronales Netz mit TensorFlow implementieren. Zusätzlich brauchen wir aber einige Funktionen aus *scikit-learn*. Wir werden den [Iris-Datensatz](https://en.wikipedia.org/wiki/Iris_flower_data_set) benutzen und versuchen mit dem neuronalen Netz die Blumen richtig zu klassifizieren.

Zum ersten wollen wir Tensorflow und einige Funktionen aus *scikit-learn* importieren. Dann laden wir den Datensatz.

**TODO: Upload pictures**

In [ ]:
import tensorflow as tf
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# Datensatz
data = load_iris()
features = data.data
labels = data.target.reshape((-1, 1))

# Klassen von Blumen zeigen
labels

Wie man hier sieht, sind die Blummenklassen mit 0, 1 oder 2 bezeichnet. In der Regel, wenn man neuronale Netze für Klassifizierungsprobleme benutzen will, hat man so viele Neuronen in der letzten Schicht wie Klassen. Wir erwarten für jede Stichprobe eine Ausgabe, die so Aussieht `[0.15, 0.70, 0.15]`. In diesem Beispiel ist die Klasse der Blume `1`, weil die zweite Zahl die größte ist.  

Wir wollen aber die Vorhersagen des neuronalen Netzes mit den echten Klassen vergleichen, deswegen wandeln wir die numerische Darstellung der Klassen zur sogenannten *One-hot encoding*. So hat man eine 3-Array als Bezeichner für jede Klasse.

Beispiel:
```
Klasse 0 -> [1, 0, 0]
Klasse 1 -> [0, 1, 0]
Klasse 2 -> [0, 0, 1]
```

Wir verwenden dafür die `OneHotEncoder` von *scikit-learn*

In [3]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(sparse=False)
enc.fit_transform(labels)

Um das Modell gut validieren zu können, sollen wir Daten verwenden, mit denen nie trainiert worden ist. Deswegen benutzen wir die `train_test_split` Funktion, um 20 Prozent der Daten als Testdaten zu nehmen. Dann merken wir uns die Anzahl von Merkmale bzw. von Klassen. In diesem Fall ist `x_size = 4` und `y_size = 3`

In [4]:
# Trainings- und Testdaten erzeugen
train_x, test_x, train_y, test_y = train_test_split(
                                features, 
                                enc.transform(labels) )

x_size = train_x.shape[1]
y_size = train_y.shape[1]

In [ ]:
X = tf.placeholder(tf.float32, shape=[None, x_size])
Y = tf.placeholder(tf.float32, shape=[None, y_size])

In [ ]:
# Neural Network Model
def hidden_layer(t_input, w_shape, activation=tf.nn.sigmoid):
    biases = tf.Variable(tf.random_normal(w_shape))
    weights = tf.Variable(tf.random_normal(w_shape))
    return activation(tf.add(biases, tf.matmul(t_input, weights)))

h_layer1 = hidden_layer(X, [x_size, 128])
h_layer2 = hidden_layer(h_layer1, [128, 128])
y_hat = hidden_layer(h_layer2, [128, y_size], tf.nn.softmax)

In [ ]:
h_layer1 = tf.layers.dense(X, 128, activation=tf.nn.sigmoid)
h_layer2 = tf.layers.dense(h_layer1, 128, activation=tf.nn.sigmoid)
y_hat = tf.layers.dense(h_layer2, y_size, activation=tf.nn.softmax)

In [ ]:
# Loss and Optimization
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
            labels=Y, logits=y_hat))

train_step = tf.train.GradientDescentOptimizer(0.005).minimize(loss)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
# Training Loop
for epoch in range(300):
    for i in range(train_x.shape[0]):
        sess.run(train_step, feed_dict={X: train_x[i:i+1], Y: train_y[i: i+1]})                                                 
        
    if (epoch % 10 == 0):
        correct_prediction = tf.equal(tf.argmax(y_hat, 1), tf.argmax(Y, 1))                                                      
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))                                                       
        print("Epoch %d: Accuraccy = %f, Loss = %f" % (                                                                          
                          epoch,                                                                                                   
                          sess.run(accuracy, feed_dict={X: test_x, Y: test_y}),                                                    
                          sess.run(loss, feed_dict={X: train_x, Y: train_y})))
    